In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np

# =========================
# Set paths (relative to current working directory)
# =========================
cwd = os.getcwd()  # Current working directory

# Input folder: preprocessed_basic
input_base = os.path.join(cwd, "preprocessed_basic")  
output_base = os.path.join(cwd, "preprocessed_dataset")  # Where enhanced images will be saved

subsets = ['train', 'val', 'test']  # If your preprocessed_basic already has subsets, otherwise just [''] for flat structure

# =========================
# Create output folders
# =========================
for subset in subsets:
    subset_input = os.path.join(input_base, subset)
    subset_output = os.path.join(output_base, subset)
    os.makedirs(subset_output, exist_ok=True)
    
    if os.path.exists(subset_input):
        classes = [cls for cls in os.listdir(subset_input) if os.path.isdir(os.path.join(subset_input, cls))]
        for cls in classes:
            os.makedirs(os.path.join(subset_output, cls), exist_ok=True)
    else:
        # For flat structure without subsets
        classes = []

# =========================
# Preprocessing Functions
# =========================
def apply_CLAHE(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

def apply_sharpen(img):
    kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    return cv2.filter2D(img, -1, kernel)

def apply_gaussian(img):
    return cv2.GaussianBlur(img, (5,5), 0)

# =========================
# Process and Save Images
# =========================
for subset in subsets:
    print(f"Processing {subset} set...")
    subset_input = os.path.join(input_base, subset)
    subset_output = os.path.join(output_base, subset)
    
    if not os.path.exists(subset_input):
        continue
    
    classes = [cls for cls in os.listdir(subset_input) if os.path.isdir(os.path.join(subset_input, cls))]
    
    for cls in classes:
        class_input = os.path.join(subset_input, cls)
        class_output = os.path.join(subset_output, cls)
        
        images = [f for f in os.listdir(class_input) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img_name in images:
            img_path = os.path.join(class_input, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224,224))
            
            # Apply all enhancements
            clahe_img = apply_CLAHE(img)
            sharpen_img = apply_sharpen(img)
            gaussian_img = apply_gaussian(img)
            
            # Save images
            base_name = os.path.splitext(img_name)[0]
            cv2.imwrite(os.path.join(class_output, f"{base_name}_clahe.jpg"), cv2.cvtColor(clahe_img, cv2.COLOR_RGB2BGR))
            cv2.imwrite(os.path.join(class_output, f"{base_name}_sharpen.jpg"), cv2.cvtColor(sharpen_img, cv2.COLOR_RGB2BGR))
            cv2.imwrite(os.path.join(class_output, f"{base_name}_gaussian.jpg"), cv2.cvtColor(gaussian_img, cv2.COLOR_RGB2BGR))
            
        print(f"  Processed class: {cls} | Total images: {len(images)}")
        
print("✅ Preprocessing complete for all subsets!")
print(f"Processed dataset saved in: {output_base}")
